In [2]:
 %load_ext nb_mypy

Version 1.0.4


In [3]:
import torch

torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [4]:
y = torch.randint(low=0, high=10, size=(2, 3))
print(y)

mean = torch.mean(y, dim = 0, dtype=torch.float)
print(mean)

tensor([[8, 6, 5],
        [2, 4, 4]])
tensor([5.0000, 5.0000, 4.5000])


In [5]:
xbow = torch.zeros(B, T, C)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, dim = 0)


print(xprev)
print(x[0])

tensor([[ 1.6455, -0.8030],
        [ 1.3514, -0.2759],
        [-1.5108,  2.1048],
        [ 2.7630, -1.7465],
        [ 1.4516, -1.5103],
        [ 0.8212, -0.2115],
        [ 0.7789,  1.5333],
        [ 1.6097, -0.4032]])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])


In [6]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [7]:
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print(a)
print(b)
print(c)


tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[7., 4.],
        [5., 0.],
        [5., 3.]])
tensor([[17.,  7.],
        [17.,  7.],
        [17.,  7.]])


In [8]:
a = torch.tril(torch.ones(3, 3))
print(a)

sum = torch.sum(a, dim=1, keepdim=True)
print(sum)

a = a / sum
print(a)

b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print(b)
print(c)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
tensor([[1.],
        [2.],
        [3.]])
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[8., 9.],
        [2., 7.],
        [3., 9.]])
tensor([[8.0000, 9.0000],
        [5.0000, 8.0000],
        [4.3333, 8.3333]])


In [9]:
a = torch.ones(2, 3, 3, dtype=torch.int64)
b = torch.randint(0, 10, (3, 2))
c = a @ b # (2, 3, 3) @ (3, 2) -> (2, 3, 2) with broadcasting of the 0 dimension

print(a)
print(b)
print(c)

tensor([[[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]]])
tensor([[4, 1],
        [6, 8],
        [6, 9]])
tensor([[[16, 18],
         [16, 18],
         [16, 18]],

        [[16, 18],
         [16, 18],
         [16, 18]]])


In [10]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim = True)

print(wei)

xbow2 = wei @ x # (T, T) @ (B, T, C) -> (B, T, C), broadcasted the B dimension
print(xbow2[0])

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [11]:
torch.allclose(xbow, xbow2)

True

In [12]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T) # intitally all tokens have zero affinity
wei = wei.masked_fill(tril == 0, float('-inf')) # future tokens have -inf affinities
wei = torch.nn.functional.softmax(wei, dim=-1)
xbow3 = wei @ x

print(xbow3[0])

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [14]:
head_size = 16

key = torch.nn.Linear(C, head_size, bias=False)
query = torch.nn.Linear(C, head_size, bias=False)
value = torch.nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x) # (B, T, 16)

wei = q @ k.transpose(-2, -1) * head_size**-0.5 # (B, T, T)
wei = wei.masked_fill(tril == 0, float('-inf')) # future tokens have -inf affinities
wei = torch.nn.functional.softmax(wei, dim=-1) 

xbow4 = wei @ v # (B, T, 16)

xbow4.shape

torch.Size([4, 8, 16])